# Thread Synchornization:

## Problems without Thread Synchronization:

## Race Condition:
### Scenario: Bus Seat Reservation
A bus has 2 available seats. Two users, Susan and Prajwal, try to reserve seats concurrently:
- Susan wants to book 2 seats.
- Prajwal wants to book 1 seat.

In [12]:
from threading import Thread, current_thread
from time import sleep

class Bus:
    def __init__(self, name, available_seats):
        self.available_seats = available_seats
        self.name = name

    def reserve(self, need_seats):
        print(f"Available seats are: {self.available_seats}")
        if self.available_seats >= need_seats:
            nm = current_thread().name
            print(f"{need_seats} seat(s) are allocated to {nm}")
            sleep(1)  # Simulate processing delay
            self.available_seats -= need_seats
        else:
            print(f"Sorry! Not enough seats for {current_thread().name}")

# Create a bus with 2 seats
b1 = Bus("Makalu Yatayat", 2)

# Create threads for Susan and Prajwal
t1 = Thread(target=b1.reserve, args=(2,), name="Susan")
t2 = Thread(target=b1.reserve, args=(1,), name="Prajwal")

# Start threads
t1.start()
t2.start()

# Wait for threads to finish
t1.join()
t2.join()


Available seats are: 2
2 seat(s) are allocated to Susan
Available seats are: 2
1 seat(s) are allocated to Prajwal


### Issues (Race Condition):
- Simultaneous Access: Both threads may check self.available_seats (2 seats) before it is updated, thinking seats are available.

- Inconsistent Results: Both threads may reserve seats, resulting in negative seat count or overselling.

## Thread Synchronization using Locks:

### 1. Solution with Mutex Lock:
- To solve the race condition and prevent data corruption, we can use a mutex lock to synchronize access to the shared counter variable. This ensures that only one thread can modify the counter at a time, preventing the threads from interfering with each other.

### acquire(blocking=True,timeout=-1):
- Blocks until the lock is available.
- Optionally accepts a timeout parameter (default is -1, meaning block indefinitely).
- Returns True when the lock is acquired, False if the timeout expires without acquiring it.

### release():
- Releases the lock, allowing other threads to acquire it.
- Must be called by the thread holding the lock.
- Raises a RuntimeError if called by a thread that doesn't hold the lock.

In [13]:
from threading import Thread, current_thread, Lock
from time import sleep

class Bus:
    def __init__(self, name, available_seats):
        self.available_seats = available_seats
        self.name = name
        self.lock = Lock()  # Create a lock for synchronization

    def reserve(self, need_seats):
        with self.lock:  # Ensure only one thread executes this block at a time
            print(f"Available seats are: {self.available_seats}")
            if self.available_seats >= need_seats:
                nm = current_thread().name
                print(f"{need_seats} seat(s) are allocated to {nm}")
                sleep(1)  # Simulate processing delay
                self.available_seats -= need_seats
            else:
                print(f"Sorry! Not enough seats for {current_thread().name}")

# Initialize the bus with 2 seats
b1 = Bus("Makalu Yatayat", 2)

# Create threads for Susan and Prajwal
t1 = Thread(target=b1.reserve, args=(2,), name="Susan")
t2 = Thread(target=b1.reserve, args=(1,), name="Prajwal")

t1.start()
t2.start()

t1.join()
t2.join()


Available seats are: 2
2 seat(s) are allocated to Susan
Available seats are: 0
Sorry! Not enough seats for Prajwal


### How the Lock Solves the Issue:
- The with self.lock ensures that only one thread executes the reserve method at a time.
- It prevents simultaneous access to the shared available_seats variable, avoiding overselling or inconsistencies.

### Disadvantages of Mutex Lock:
#### i. Cannot be Acquired Multiple Times by the Same Thread:

- A mutex lock can only be acquired once by the thread that holds it. If the same thread tries to acquire it again (in nested calls), it will cause a deadlock.
- This makes it unsuitable for recursive or reentrant operations that require the thread to enter the locked section multiple times.

### Why we use multiple locks on same resource?
- Multiple locks on the same resource can happen due to mistakes by the developer, especially if they are unaware of how other parts of the code are locking the same resource. This can lead to issues like deadlocks or performance inefficiencies.

### 2. Solution with Rlock:

In [ ]:
# Problem of Mutex lock

import threading

# Shared resource
counter = 0

# Create a mutex lock
mutex = threading.Lock()

def increment():
    global counter
    mutex.acquire()  # Acquire lock for the first time
    counter += 1
    print(f"Counter after increment: {counter}")
    
    mutex.acquire()  # Attempt to acquire the lock again (This will cause a deadlock)
    counter += 1
    print(f"Counter after second increment: {counter}")
    mutex.release()  # Release lock
    mutex.release()  # Release lock again

# Start a thread
thread = threading.Thread(target=increment)
thread.start()
thread.join()

print(f"Final Counter Value: {counter}")


Counter after increment: 1


### Issue:
- When the increment() function runs, it acquires the lock and increments the counter.
- After the first counter += 1, the function attempts to acquire the same lock again with mutex.acquire().
- Mutex lock does not allow the same thread to acquire the lock again, and the second mutex.acquire() causes the thread to deadlock.
- The thread is waiting for itself to release the lock, and the program hangs indefinitely.

### How RLock Solves the Issue:
- The reentrant lock (RLock) allows the thread to acquire the same lock multiple times without causing a deadlock. It works by tracking the number of times a thread has acquired the lock. The thread can call acquire() multiple times, and it needs to call release() the same number of times to unlock it.

In [1]:
import threading

# Shared resource
counter = 0

# Create a reentrant lock (RLock)
rlock = threading.RLock()

def increment():
    global counter
    rlock.acquire()  # Acquire lock for the first time
    counter += 1
    print(f"Counter after increment: {counter}")
    
    rlock.acquire()  # Re-enter and acquire the lock again (No deadlock)
    counter += 1
    print(f"Counter after second increment: {counter}")
    
    rlock.release()  # Release lock
    rlock.release()  # Release lock again (releasing the same number of times)

# Start a thread
thread = threading.Thread(target=increment)
thread.start()
thread.join()

print(f"Final Counter Value: {counter}")


Counter after increment: 1
Counter after second increment: 2
Final Counter Value: 2


### key points:
- Matching acquire and release: For each acquire(), there must be a corresponding release().
- Cannot release more times than acquired: Calling release() more times than acquire() raises a RuntimeError.
- Cannot release without acquiring: You cannot release() a lock without first calling acquire().
- Multiple acquires by same thread: The thread can acquire the lock multiple times, but must release it the same number of times.
- Thread ownership: Only the thread that acquired the lock can release it; other threads cannot release it.

### Limitation of Locks and RLocks:
- Single Thread Access: Locks (including RLocks) only allow one thread to acquire the lock at any given time. This means only one thread can execute the critical section of code while others are blocked, limiting concurrency.

### 3. Solution with Semaphore:
- A semaphore is a synchronization primitive used to control access to a shared resource by multiple threads in a concurrent system. It maintains a counter, which indicates the number of available resources or the number of threads that can access the critical section.

### 3.1 Unbounded Semaphore (No Limit):
#### What is it?
- An unbounded semaphore doesn’t have a fixed limit. It starts with an initial number of tickets, but you can keep releasing tickets as long as you want.
- Example: Imagine you have 3 tickets to start with, but after a thread finishes, you can release as many tickets as you like, and new threads can keep acquiring them.

#### How it works:
- If you initialize the semaphore with 3 permits, threads will be able to acquire permits like before.
- But if you release more tickets than you started with, the semaphore counter can grow. It doesn't throw an error for releasing more tickets than initially acquired.

In [2]:
from threading import Thread, Semaphore, current_thread
import time

# Unbounded semaphore with 3 permits
semaphore = Semaphore(3)

def use_resource():
    current_thread_name = current_thread().name
    print(f"{current_thread_name} is trying to acquire a ticket...")
    
    semaphore.acquire()  # Try to acquire a permit (ticket)
    print(f"{current_thread_name} is using the resource.")
    time.sleep(1)  # Simulate using the resource
    print(f"{current_thread_name} is releasing the ticket.")
    
    semaphore.release()  # Release the permit (ticket)

# Create 5 threads (more than the permits)
threads = [Thread(target=use_resource, name=f"Thread-{i}") for i in range(5)]

# Start the threads
for t in threads:
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

print("All threads have finished.")


Thread-0 is trying to acquire a ticket...Thread-1 is trying to acquire a ticket...
Thread-1 is using the resource.

Thread-0 is using the resource.
Thread-2 is trying to acquire a ticket...
Thread-2 is using the resource.
Thread-3 is trying to acquire a ticket...
Thread-4 is trying to acquire a ticket...
Thread-1 is releasing the ticket.Thread-2 is releasing the ticket.
Thread-0 is releasing the ticket.
Thread-3 is using the resource.
Thread-4 is using the resource.

Thread-3 is releasing the ticket.Thread-4 is releasing the ticket.

All threads have finished.


### Explanation:
- Semaphore starts with 3 tickets.
- 3 threads acquire tickets, so the counter goes from 3 → 0.
- After using the resource, each thread releases a ticket.
- After releasing 3 tickets, the semaphore counter becomes 3 again.
- If 4 tickets are released (e.g., releasing a 4th ticket when the counter is 0), the semaphore counter goes to 4.

### 3.2 Bounded Semaphore (Fixed Limit):
#### What is it?
- A bounded semaphore is a semaphore that has a fixed maximum limit on how many threads can access the resource at the same time.
- Example: Imagine you have 3 tickets (permits) for 3 people to access a printer. Once 3 people are using the printer, the 4th person has to wait because all tickets are taken.

#### How it works:
- If you initialize a bounded semaphore with 3 permits, only 3 threads can access the resource at once.
- Once the limit is reached, new threads will have to wait until someone finishes and releases a ticket.

In [10]:
from threading import Thread, BoundedSemaphore, current_thread
import time

# Unbounded semaphore with 3 permits
semaphore = BoundedSemaphore(3)

def use_resource():
    # Correct way to get the thread name
    current_thread_name = current_thread().name
    print(f"{current_thread_name} is trying to acquire a ticket...")
    
    semaphore.acquire()  # Try to acquire a permit (ticket)
    print(f"{current_thread_name} is using the resource.")
    time.sleep(1)  # Simulate using the resource
    print(f"{current_thread_name} is releasing the ticket.")
    
    semaphore.release()  # Release the permit (ticket)

# Create 5 threads (more than the permits)
threads = [Thread(target=use_resource, name=f"Thread-{i}") for i in range(5)]

# Start the threads
for t in threads:
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

print("All threads have finished.")


Thread-0 is trying to acquire a ticket...Thread-1 is trying to acquire a ticket...
Thread-1 is using the resource.
Thread-2 is trying to acquire a ticket...
Thread-2 is using the resource.

Thread-0 is using the resource.
Thread-3 is trying to acquire a ticket...
Thread-4 is trying to acquire a ticket...
Thread-1 is releasing the ticket.Thread-0 is releasing the ticket.
Thread-2 is releasing the ticket.
Thread-3 is using the resource.
Thread-4 is using the resource.

Thread-3 is releasing the ticket.
Thread-4 is releasing the ticket.
All threads have finished.


### Explanation:
- Semaphore starts with a fixed number of tickets (e.g., 3).
- Threads acquire tickets: The counter decreases as threads acquire tickets. For example, 3 threads acquire tickets, so the counter goes from 3 → 0.
- Once the semaphore reaches 0, no more threads can acquire a ticket. They must wait until a ticket is released.
- After a thread finishes using the resource, it releases a ticket. The counter increases by 1 for each release.
- If a thread tries to release more tickets than the initial limit (3), the semaphore will not allow it, and the ticket count will remain within the initial limit.

## How semaphore eliminate race condition?
#### What Semaphore Can Do:
- Control access: Semaphore allows multiple threads to access a shared resource concurrently, but it limits the number of threads that can access the resource at any given time.
- Concurrency management: By limiting the number of threads, it can prevent the system from being overwhelmed by too many threads trying to access a resource simultaneously.
#### What Semaphore Cannot Do:

- Prevent race conditions: Semaphore by itself does not guarantee that multiple threads won't modify shared data concurrently, leading to race conditions. Multiple threads can still access and modify shared data simultaneously unless mutual exclusion (via locks) is implemented.

### Eliminating Race Conditions with Semaphore + Lock
To eliminate race conditions while using semaphores, semaphores control the concurrency (how many threads can access a resource), and locks (like Lock, RLock) ensure mutual exclusion when threads modify the shared resource.

- Semaphore: Controls how many threads can access the critical section at a time.
- Lock: Ensures that only one thread can modify the shared resource at a time, thus preventing race conditions.

In [11]:
import threading
import time

# Shared resource (counter)
counter = 0

# Semaphore allowing up to 3 threads to access the shared resource
semaphore = threading.Semaphore(3)

# Lock to ensure mutual exclusion when modifying the shared resource
lock = threading.Lock()

def increment():
    global counter
    semaphore.acquire()  # Limit the number of threads accessing the resource

    with lock:  # Ensure only one thread modifies the counter at a time
        current_value = counter
        time.sleep(0.1)  # Simulate some processing
        counter = current_value + 1
        print(f"Counter incremented by {threading.current_thread().name}. New counter: {counter}")

    semaphore.release()  # Allow another thread to access the resource

# Create and start multiple threads
threads = []
for i in range(5):
    thread = threading.Thread(target=increment, name=f"Thread-{i+1}")
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

print(f"Final Counter Value: {counter}")


Counter incremented by Thread-1. New counter: 1
Counter incremented by Thread-2. New counter: 2
Counter incremented by Thread-3. New counter: 3
Counter incremented by Thread-4. New counter: 4
Counter incremented by Thread-5. New counter: 5
Final Counter Value: 5
